<a href="https://colab.research.google.com/github/Karen-20/Karen/blob/master/KPD_supplementary_material_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# This is formatted as code
```


# Supplementary material

Authors:  S. Kokabisaghi and E.J. Pauwels 

Affiliation:  CWI, Amsterdam

Link to paper on Arxiv

Date:   March 2020

Version:  1.0

Notes: 

  1. Changed payoff into util since the result does already incorporate gamma and is therefore a utility, not a payoff. 
  2. Simplified the MZ table by using sigma = 1, original table is now called mz_table_full


In [0]:
# Import required packages
import numpy as np
from sympy import *
import pandas as pd # for table

import matplotlib.pyplot as plt #Required packages for plotting 
from matplotlib import rcParams
plt.rcParams["font.weight"] = "bold"# Bold text in plot
plt.rcParams["axes.labelweight"] = "bold"

**Structure:**


1.   Compute expected utility for each types of agents using events table [Utility Computation](#scrollTo=rkkPIYMHwYuD&line=5&uniqifier=1)
2.   Compute utility's endpoints (A, B, C, D)  [ Utility 's End points](#scrollTo=Y8oBU4Z0Y3ll&line=6&uniqifier=1)
3.   Compute risk aversion thresholds  [Risk Aversion](#scrollTo=CLc9mlt42esO)

*   Compute optimal utility  [$u^*(p)$](#scrollTo=Q_GfpxsCQEk2&line=18&uniqifier=1)
*   Compute $\frac{du*}{dp}|p$ and check the [Conditions](#scrollTo=CLc9mlt42esO) 
*    Compute the threshold from sure to probabilistic sniping [$\overline{\gamma}_K$](#scrollTo=sDM28-BeLmRA)

*  Compute the threshold from probabilistic to no sniping 
 [$\overline{\gamma}_L$](#scrollTo=eiGobNsHtc-R&line=3&uniqifier=1)
---
4. [Check KPD equations numerically ](#scrollTo=_qJ8lNPw3YlU&line=3&uniqifier=1)














 

**Notations and endpoints for each agent's utility:** (KPD: section 2.2)


1.   $U_B(s=0) = A \quad$   and  $\quad U_B(s=\sigma) = B $
2.   $U_M(s:0)$ = C  and $U_M(s:\sigma)$ = D
 $$ U_B(s)=(1-\frac{s}{\sigma}) A + (\frac{1}{s}) B \\
U_M(s) = (1-\frac{s}{\sigma}) C+ (\frac{1}{s}) D$$












# 0. Introduce symbols and values

In this notebook we will perform both symbolic and numerical calculations, for which we here introduce the symbols and some values. 

In [0]:
# Introduce the primary parameters as Symbol

from sympy.abc import s, alpha, mu, delta, gamma, sigma, p, H  

# introduce the symbols for secondary parameters (are functions of the primary parameters)

from sympy.abc import beta, g,h,q, theta 
alphabar = symbols('alphabar')
mubar = symbols('mubar')
thetabar = symbols('thetabar')
m = 1-mubar

#  Introduce some numerical values for the primary paramaters and use them 
#  to compute the corresponding values for the secondary (i.e. derived) parameters

alpha_val = .15
mu_val  = .005
delta_val = .5
H_val = 10
gamma_val = 1.5
sigma_val =1
s_val = .45
#
beta_val = alpha_val/(alpha_val + mu_val)
alphabar_val = alpha_val * delta_val /2
mubar_val = mu_val * delta_val/2
q_val = gamma_val -1
beta_val = alpha_val/(alpha_val+mu_val)
theta_val = 2*alpha_val*mu_val/(alpha_val+mu_val)
thetabar_val = theta_val*delta_val/2
m_val = 1- mubar_val

#  Create a dictionary that makes the appropriate substitutions for numerical experiments 


dict_subs_numerical_vals = {alpha: alpha_val, mu : mu_val, delta :delta_val, H : H_val, gamma :gamma_val, q: q_val, 
                            beta: beta_val, mubar:mubar_val, theta :theta_val, thetabar :thetabar_val, alphabar: alphabar_val, m:m_val}

print(dict_subs_numerical_vals)


{alpha: 0.15, mu: 0.005, delta: 0.5, H: 10, gamma: 1.5, q: 0.5, beta: 0.9677419354838709, mubar: 0.00125, theta: 0.00967741935483871, thetabar: 0.0024193548387096775, alphabar: 0.0375, -mubar + 1: 0.99875}


# 1. Define the utility tables

Encoding the table 1 in MZ paper, which we will use to compute the expected utility for both the market maker ($U_M$) and bandit ($U_B$)

In [0]:


#  Encode the pay-off table (MZ paper p. 1201) as a list of dictionaries: 
#-------------------------------------------------------------------------- 
#  REMARKS:  
#  -  The events in the table below are mutually exclusive.  This makes it possible to 
#     decide which events yield a positive or negative outcome. In the latter case we 
#     need to multiply by gamma (risk aversion factor) 
#  -  Notice that we represent the fraction 1/2 as Rational(1,2) in order to take streamline computations with fractions! 
#  -  NG-LA = trigger =  good news, 2nd event = Liquidity trader on Ask etc.
#  The subscript _rwy means that the payoff is valid if the agent is the racewinner. 
#   Similarly _rwn means that the agent lost the race.    
# 
#  This table is essenially taken from the MZ paper


mz_table_full = [{'eventcode':'NG-LA' , 'prob_trigger':beta, 'prob_2nd_event':Rational(1,2)*mubar, 'util_mm_rwy':gamma*(s-sigma), 'util_mm_rwn':gamma*(s-sigma)  , 'util_bandit_rwy': 0 , 'util_bandit_rwn': 0},
            {'eventcode':'NG-LB' , 'prob_trigger':beta, 'prob_2nd_event': Rational(1,2)*mubar, 'util_mm_rwy':(s+sigma), 'util_mm_rwn':2*s,'util_bandit_rwy' : (sigma-s),'util_bandit_rwn': 0},
            {'eventcode':'NG-NG' , 'prob_trigger':beta, 'prob_2nd_event':Rational(1,2)*alphabar, 'util_mm_rwy': 0 , 'util_mm_rwn': gamma*(s -2*sigma) ,'util_bandit_rwy': 2*sigma-s,'util_bandit_rwn': 0} ,           
            {'eventcode':'NG-NB' , 'prob_trigger':beta, 'prob_2nd_event':Rational(1,2)*alphabar, 'util_mm_rwy': 0,'util_mm_rwn': s, 'util_bandit_rwy': -s * gamma, 'util_bandit_rwn': 0},            
            {'eventcode':'NG-NO' , 'prob_trigger':beta, 'prob_2nd_event':Rational(1,2)*(1-2*(alphabar+mubar)), 'util_mm_rwy':0, 'util_mm_rwn': gamma*(s -sigma), 'util_bandit_rwy':sigma-s, 'util_bandit_rwn':0},
            {'eventcode':'NB-LA' , 'prob_trigger':beta, 'prob_2nd_event':Rational(1,2)*mubar, 'util_mm_rwy': s+sigma,'util_mm_rwn':2*s, 'util_bandit_rwy':sigma -s,'util_bandit_rwn': 0 },
            {'eventcode':'NB-LB' , 'prob_trigger':beta, 'prob_2nd_event':Rational(1,2)*mubar, 'util_mm_rwy': gamma*(s-sigma), 'util_mm_rwn':gamma*(s-sigma), 'util_bandit_rwy':0, 'util_bandit_rwn': 0},    
            {'eventcode':'NB-NG' , 'prob_trigger':beta, 'prob_2nd_event':Rational(1,2)*alphabar, 'util_mm_rwy':0 ,'util_mm_rwn': s, 'util_bandit_rwy': -s *gamma, 'util_bandit_rwn': 0 },
            {'eventcode':'NB-NB' , 'prob_trigger':beta, 'prob_2nd_event':Rational(1,2)*alphabar, 'util_mm_rwy':0 , 'util_mm_rwn': gamma*(s-2*sigma) , 'util_bandit_rwy':(2*sigma-s) , 'util_bandit_rwn':0},
            {'eventcode':'NB-NO' , 'prob_trigger':beta, 'prob_2nd_event':Rational(1,2)*(1-2*(alphabar+mubar)), 'util_mm_rwy': 0, 'util_mm_rwn':  gamma*(s-sigma), 'util_bandit_rwy':(sigma-s) , 'util_bandit_rwn': 0},
            {'eventcode':'LA-LA' , 'prob_trigger':(1-beta),'prob_2nd_event':Rational(1,2)*mubar, 'util_mm_rwy':s , 'util_mm_rwn': s, 'util_bandit_rwy':0 , 'util_bandit_rwn': 0},
            {'eventcode':'LA-LB' , 'prob_trigger':(1-beta),'prob_2nd_event':Rational(1,2)*mubar, 'util_mm_rwy': 2*s, 'util_mm_rwn': 2*s,  'util_bandit_rwy':0 , 'util_bandit_rwn': 0},
            {'eventcode':'LA-NG' , 'prob_trigger':(1-beta),'prob_2nd_event':Rational(1,2)*alphabar, 'util_mm_rwy': gamma*(s-sigma), 'util_mm_rwn': gamma*(s-sigma),  'util_bandit_rwy':0 , 'util_bandit_rwn': 0},
            {'eventcode':'LA-NB' , 'prob_trigger':(1-beta),'prob_2nd_event':Rational(1,2)*alphabar, 'util_mm_rwy': s +sigma, 'util_mm_rwn': s+ sigma,  'util_bandit_rwy':0 , 'util_bandit_rwn': 0},
            {'eventcode':'LA-NO' , 'prob_trigger':(1-beta),'prob_2nd_event':Rational(1,2)*(1-2*(alphabar+mubar)), 'util_mm_rwy': s, 'util_mm_rwn': s,  'util_bandit_rwy':0 , 'util_bandit_rwn': 0},
            {'eventcode':'LB-LA' , 'prob_trigger':(1-beta),'prob_2nd_event':Rational(1,2)*mubar, 'util_mm_rwy': 2*s, 'util_mm_rwn': 2*s,  'util_bandit_rwy':0 , 'util_bandit_rwn': 0},
            {'eventcode':'LB-LB' , 'prob_trigger':(1-beta),'prob_2nd_event':Rational(1,2)*mubar, 'util_mm_rwy': s, 'util_mm_rwn': s,  'util_bandit_rwy':0 , 'util_bandit_rwn': 0},
            {'eventcode':'LB-NG' , 'prob_trigger':(1-beta),'prob_2nd_event':Rational(1,2)*alphabar, 'util_mm_rwy': s+ sigma, 'util_mm_rwn':  s+ sigma,  'util_bandit_rwy':0 , 'util_bandit_rwn': 0},
            {'eventcode':'LB-NB' , 'prob_trigger':(1-beta),'prob_2nd_event':Rational(1,2)*alphabar, 'util_mm_rwy': gamma*(s -sigma), 'util_mm_rwn':  gamma*(s -sigma),  'util_bandit_rwy':0 , 'util_bandit_rwn': 0},
            {'eventcode':'LB-NO' , 'prob_trigger':(1-beta),'prob_2nd_event':Rational(1,2)*(1-2*(alphabar+mubar)), 'util_mm_rwy':s, 'util_mm_rwn':  s,  'util_bandit_rwy':0 , 'util_bandit_rwn': 0}]



Table =pd.DataFrame(mz_table_full)
pd.set_option('display.max_columns', 8)
pd.set_option('display.width', 180)
print(Table)

    



   eventcode prob_trigger           prob_2nd_event        util_mm_rwy          util_mm_rwn util_bandit_rwy  util_bandit_rwn
0      NG-LA         beta                  mubar/2  gamma*(s - sigma)    gamma*(s - sigma)               0                0
1      NG-LB         beta                  mubar/2          s + sigma                  2*s      -s + sigma                0
2      NG-NG         beta               alphabar/2                  0  gamma*(s - 2*sigma)    -s + 2*sigma                0
3      NG-NB         beta               alphabar/2                  0                    s        -gamma*s                0
4      NG-NO         beta  -alphabar - mubar + 1/2                  0    gamma*(s - sigma)      -s + sigma                0
5      NB-LA         beta                  mubar/2          s + sigma                  2*s      -s + sigma                0
6      NB-LB         beta                  mubar/2  gamma*(s - sigma)    gamma*(s - sigma)               0                0
7      N

## Note:  Simplify table by setting sigma = 1

Since the jumpsize $\sigma$ (sigma) is in fact a scaling parameter we can, without loss of generality, fix it to one:  (sigma = 1). 
For the rest of this notebook, we will use this convention. 




In [0]:
# Make a copy of the table and substitute sigma = 1

mz_table = mz_table_full.copy()

for d in mz_table: 
  for x,y in d.items():
    #  substitution only works for expressions, exclude the rest
    if type(y) != str  and type(y) != int:
      y_new = y.subs({sigma:1})
      d.update({x:y_new})


Table =pd.DataFrame(mz_table)
pd.set_option('display.max_columns', 8)
pd.set_option('display.width', 180)
print("======++++++++++++++++++++++= Utility table (simplified for sigma = 1) =======================")
print(' ')
print(Table)



======++++++++++++++++++++++= Utility table (simplified for sigma = 1) =======================
 
   eventcode prob_trigger           prob_2nd_event    util_mm_rwy    util_mm_rwn util_bandit_rwy  util_bandit_rwn
0      NG-LA         beta                  mubar/2  gamma*(s - 1)  gamma*(s - 1)               0                0
1      NG-LB         beta                  mubar/2          s + 1            2*s          -s + 1                0
2      NG-NG         beta               alphabar/2              0  gamma*(s - 2)          -s + 2                0
3      NG-NB         beta               alphabar/2              0              s        -gamma*s                0
4      NG-NO         beta  -alphabar - mubar + 1/2              0  gamma*(s - 1)          -s + 1                0
5      NB-LA         beta                  mubar/2          s + 1            2*s          -s + 1                0
6      NB-LB         beta                  mubar/2  gamma*(s - 1)  gamma*(s - 1)               0         

# 2. Compute expected utility $EU_B$ for **bandit**


To compute the expected utility we successively condition on the first 
(i.e. Trigger) and second event. 



## 2.0  Preamble: probabilities $h(p)$ and $g(p)$

The expected utilities for bandit and market maker depend on three probabilities which we will introduce here.  For the detailed 
computation of these quantities we refer to another notebook ??? 

When the first event (i.e. trigger event) in the MZ game is the arrival 
of a news event, the market maker (MM) will race to update his stale quotes. 
The bandits however will only enter the race with probability $p$. 
This probability $p$ determines the values of two dependent probabilities: 
 

  1. $h(p) = P(\mbox{MM will lose the race})$
  2. $g(p) = P(\mbox{bandit will win race} \,| \, \mbox{he enters the race})$



##  2.1  Conditioning on first (i.e. trigger) event

The bandit will only receive payoff if he enters and wins the race. 
For this to happen, a number of conditions need to be fulfilled: 

1.   there should be a race (i.e. the trigger event is news (prob = $\beta := \alpha/(\alpha+\mu)$)
2.   the bandit needs to enter the race (prob = $p$), and 
3.   he needs to win the race (prob = $g(p)$).

Therefore: 

  $$ \begin{eqnarray*}
     EU_B &=& E(U_B |\quad Tr = N)\, P(Tr = N) \nonumber \\
     &=&   E(U_B |\quad Tr = N) \, \beta  \\
     &=&   E(U_B|\quad Tr = N\, \&\, \mbox{bandit enters race}) 
     P\{\mbox{bandit enters race}\}\,\beta\\
     &=&  E(U_B |\quad \mbox{bandit enters and wins race}) 
     P\{\mbox{bandit wins race} |\quad \mbox{he races} \} \, p \beta\nonumber\\
     & =& E(U_B |\quad \mbox{bandit enters and wins race}) \, g(p) p \beta
      \end{eqnarray*}$$
 From this we conclude:

$$ EU_B(s,p) =  \beta g(p) p\, E(U_B \,  |\,\mbox{bandit enters and wins race})$$

----------
## 2.2  Conditioning on second event

To compute the conditional expectation in the RHS we need to condition on the second event (either additional news, or the arrival of LT or nothing). 
If we use * to indicate a wildcard we can rewrite this conditial mean as: 

$$  
E(U_B \,|\, \mbox{bandit enters and wins race'} ) =  
E(\mbox{util_bandit_rwy} \,|\, \mbox{eventcode = 'N*-**'} ) 
$$

The RHS can be computed by summing over all the different $\mbox{'N*-**-rwy'}$ events 
(weighted with corresponding probabilities).  This is reported in the 
variable EU_B_given_bandit_races_and_wins below. 

---
## 2.3  Compute the endpoints of the line segments

Next, compute the values for EU_B for s=0 and s=sigma=1 
(this checks out with what is found in the paper, see eq????)

$$ 
  \begin{array}{lclclcl}
 A &:=& EU_B(s=0) &=& \beta p g(p) (1-\overline{\mu}) &=&  m \,(\beta p  g(p))\\
 B &:=& EU_B(s=1)  &=& -\overline{\alpha}(\gamma-1)  \beta p g(p) 
 &=& -\overline{\alpha}q \,(\beta  p g(p))
\end{array}
$$
 


In [0]:

EU_B_given_bandit_races_and_wins = symbols('EU_B_given_bandit_races_and_wins')

EU_B_given_bandit_races_and_wins = 0  # initialisation 

# Cycle over all events for which trigger event is news (good or bad), ie. eventcode = 'N*-**'
for d in mz_table: 
  if d['eventcode'][0] == 'N' : 
    EU_B_given_bandit_races_and_wins += d['util_bandit_rwy']*d['prob_2nd_event']

print (' ')
print('===== Conditional expectation for bandit ==============================')
print('EU_B_given_bandit_races_and_wins =  ', EU_B_given_bandit_races_and_wins)

# Combine all the conditional expectations to compute EU_B
EU_B = beta*p*g*EU_B_given_bandit_races_and_wins

print (' ')
print('===== Expected utility for bandit  =========================')
print('EU_B = ', EU_B)

#  Since the utility is linear in spread s it suffices to specify the values 
#  in two points s = 0 and s = 1 (recall sigma = 1 without loss of generality)

A = EU_B.subs({s:0})
B = EU_B.subs({s:1})
#  In order to further simplify:
A = A.collect(beta)
B = B.collect(beta).collect(alphabar) 

print(' ')
print('======= Specifying end points A=EU(s=0) and B=EU(s=1) (cf. eqs 9 and 10 ??? in paper!)=====')
print('A = EU_B(s=0) = ',A)
pprint('A = '), pprint(A)
print('B = EU_B(s=1) = ',B)
pprint('B = '), pprint(B)

 
===== Conditional expectation for bandit ==============================
EU_B_given_bandit_races_and_wins =   -alphabar*gamma*s + alphabar*(-s + 2) + mubar*(-s + 1) + 2*(-s + 1)*(-alphabar - mubar + 1/2)
 
===== Expected utility for bandit  =========================
EU_B =  beta*g*p*(-alphabar*gamma*s + alphabar*(-s + 2) + mubar*(-s + 1) + 2*(-s + 1)*(-alphabar - mubar + 1/2))
 
======= Specifying end points A=EU(s=0) and B=EU(s=1) (cf. eqs 9 and 10 ??? in paper!)=====
A = EU_B(s=0) =  beta*g*p*(-mubar + 1)
A = 
β⋅g⋅p⋅(-μ̅ + 1)
B = EU_B(s=1) =  alphabar*beta*g*p*(-gamma + 1)
B = 
α̅⋅β⋅g⋅p⋅(-γ + 1)


(None, None)

# 3. Compute expected utility $EU_M$ for **market maker** 
Since $P(Tr=N) = \beta :=\alpha/(\alpha+\mu) $ while $P(Tr=LT) =  1-\beta$ we 
get: 

$$ EU_M =  \beta\, E(U_M \,| \, Tr=N) + (1-\beta)\,E(U_M \,|\, Tr=LT)$$

Notice that we are spefically interested in the end points: 

$$  C := EU_M(s = 0) \quad \quad \mbox{and} \quad \quad 
D := EU_M(s=1)   $$

We now compute the different conditional expectations by conditioning on 
additional events: 







---

## 3.1  Trigger event is News

If trigger event is **News**, condition on outcome of the ensuing race. 
Recall that if individual HFTs enter the race with probability $p$, 
the market maker will lose the race with probability $h(p)$ 
(for an explicit computation of $h(p)$ see ?????).  Hence, 

$$\begin{eqnarray*}
 E(U_M \,| \, Tr = N) &=& E(U_M \, | \, Tr = N \,\, \& \,\,
 \mbox{MM loses race})\,  h(p) \\
&+& E(U_M \, | \,  Tr = N \,\,\& \,\, \mbox{MM wins race} ) 
 (1-h(p))
\end{eqnarray*}$$

Next, we express the conditional expectations in the RHS in terms of the corresponding event code (using * as a wild card): 

$$
 E(U_M \, | \, Tr = N \,\, \& \,\,
 \mbox{MM races and wins race}) = E(\mbox{util_mm_rwy} \,| \, \mbox{eventcode = 'N*-**'})
$$


$$
 E(U_M \,| \, Tr = N \,\, \& \,\,
 \mbox{MM races and loses race}) = E(\mbox{util_mm_rwn} \,| \, \mbox{eventcode = 'N*-**'})
$$


From these expansions we learn that: 


$$
\begin{array}{lclcl}
 \mbox{C_MM_races_and_wins} & := & E_{s=0}(U_M \,| \,  Tr = N \,\, \& \,\,
 \mbox{MM races and wins race}) &=& -\bar{\mu}q  \\
 %
 \mbox{D_MM_races_and_wins} &:=& E_{s=1}(U_M \, | \,  Tr = N \,\, \& \,\,
 \mbox{MM races and wins race})& = & 2 \bar{\mu}  \\
 %
\mbox{C_MM_races_and_loses} &:=& E_{s=0}(U_M \,| \, Tr = N \,\, \& \,\,
 \mbox{MM races and loses race}) &=&  -(1-\bar{\mu})(q+1) = -m\gamma  \\
%
 \mbox{D_MM_races_and_loses} &:=& E_{s=1}(U_M \, | \,  Tr = N \,\, \& \,\,
 \mbox{MM races and loses race}) &=& 2\overline{\mu} - \overline{\alpha} q
\end{array} 
$$


Since the utility is linear in s,  we can combine these results to obtain 
the corresponding utility function for values of s: 


(KPD.eq.14) $$ E(U_M \, | \,  Tr = N \,\, \& \,\, \mbox{MM wins race})=  \overline{\mu}(2s - q(1-s)) $$


(KPD.eq.13) $$ E(U_M \, | \, Tr = N \,\, \& \,\,
 \mbox{MM loses race})  =  (2  \overline{\mu}-\overline{\alpha} q) s 
 -m\gamma(1-s)$$



In [0]:
# Compute the expected utility for the Market maker (EU_M) by cycling over the dictionaries in the MZ-table above

#  Compute the utilities for sub-events,
#---------------------------------------
# When Trigger is news:

EU_M_given_MM_races_and_wins = symbols('EU_MM_given_MM_races_and_wins')
EU_M_given_MM_races_and_loses = symbols('EU_MM_given_MM_races_and_loses')

EU_M_given_MM_races_and_wins = 0  # initialisation
EU_M_given_MM_races_and_loses = 0  # initialisation

# Cycle over all 'N*-**' events (trigger event is news)
# and add in both cases (win/lose) the payoff (weighted with probability)
for d in mz_table:
  if d['eventcode'][0] == 'N': #  
    EU_M_given_MM_races_and_wins  += d['util_mm_rwy']*d['prob_2nd_event']
    EU_M_given_MM_races_and_loses += d['util_mm_rwn']*d['prob_2nd_event']

print('===================================')
EU_M_given_MM_races_and_wins = EU_M_given_MM_races_and_wins.cancel().collect(s).subs({gamma:q+1}).simplify()
EU_M_given_MM_races_and_loses = EU_M_given_MM_races_and_loses.subs({gamma:q+1}).cancel().collect(s).simplify()

print('EU_MM_given_MM_races_and_wins =  ', EU_M_given_MM_races_and_wins)
print('EU_MM_given_MM_races_and_loses =  ', EU_M_given_MM_races_and_loses)

#  We only need these values at s=0 and s=1

C_MM_races_and_wins = EU_M_given_MM_races_and_wins.subs({s:0})
D_MM_races_and_wins = EU_M_given_MM_races_and_wins.subs({s:1})
C_MM_races_and_loses = EU_M_given_MM_races_and_loses.subs({s:0})
D_MM_races_and_loses = EU_M_given_MM_races_and_loses.subs({s:1})

C_MM_races_and_loses = factor(C_MM_races_and_loses)
C_MM_races_and_loses = C_MM_races_and_loses.subs({mubar:1-m})

print('C_MM_races_and_wins = ', C_MM_races_and_wins)
print('D_MM_races_and_wins = ', D_MM_races_and_wins)
print('C_MM_races_and_loses = ', C_MM_races_and_loses)
print('D_MM_races_and_loses = ', D_MM_races_and_loses)


# Combine all the conditional expectations to compute EU_M_news
EU_M_news = EU_M_given_MM_races_and_wins * (1-h) + EU_M_given_MM_races_and_loses * h
print('-----------------------------------')
print('E(U_M| Tr=News) = ', EU_M_news)

C_MM_news =  C_MM_races_and_wins*(1-h) + C_MM_races_and_loses*h
D_MM_news =  D_MM_races_and_wins*(1-h) + D_MM_races_and_loses*h

print('C_MM_news = ', C_MM_news)
print('D_MM_news = ', D_MM_news)



EU_MM_given_MM_races_and_wins =   mubar*(-q + s*(q + 2))
EU_MM_given_MM_races_and_loses =   mubar*q + mubar - q + s*(-alphabar*q - mubar*q + mubar + q + 1) - 1
C_MM_races_and_wins =  -mubar*q
D_MM_races_and_wins =  2*mubar
C_MM_races_and_loses =  (mubar - 1)*(q + 1)
D_MM_races_and_loses =  -alphabar*q + 2*mubar
-----------------------------------
E(U_M| Tr=News) =  h*(mubar*q + mubar - q + s*(-alphabar*q - mubar*q + mubar + q + 1) - 1) + mubar*(-h + 1)*(-q + s*(q + 2))
C_MM_news =  h*(mubar - 1)*(q + 1) - mubar*q*(-h + 1)
D_MM_news =  h*(-alphabar*q + 2*mubar) + 2*mubar*(-h + 1)


---
## 3.2 Trigger event is arrival of liquidity trader (LT)

When the trigger event is the arrival of a liquidity trader (LT), there is no race and the utility of the market maker is independent of the outcome of the race.  So we can pick either util_mm_rwy or util_mm_rwn.

$$ E(U_M \, | \, Tr = LT) = E(\mbox{util_mm_rwy} \,|\, \mbox{eventcode = 'L*-**'} )$$

Again we are only interested in the endpoints of the function: 

$$ \mbox{C_MM_LT} =  E_{s=0}(U_M \, | \, Tr = LT) = - \bar{\alpha}q $$

and 
$$  \mbox{D_MM_LT} =  E_{s=1}(U_M \, | \, Tr = LT) = 1 +\overline{\mu}$$

Since we know that function is linear in s, we get: 
(KPD.eq.11)
$$ E(U_M| \, Tr = LT)  = (1+ \overline{\mu} )s - \overline{\alpha} q (1-s)$$

---



In [0]:
#---------------------------------------
# Trigger is arrival liquidity trader (LT)
# In this case, there is no sniping when the event is LT arrival. So payoff for MM is the same. 
print('Expected MM Utility when trigger event is LT arrival')

EU_M_given_trigger_LT_arrival = 0
#  Cycle over all LT events
for d  in mz_table:
  if d['eventcode'][0]=='L' :
    EU_M_given_trigger_LT_arrival += d['util_mm_rwn']*d['prob_2nd_event']

# Simplify
EU_M_given_trigger_LT_arrival = EU_M_given_trigger_LT_arrival.subs({gamma:q+1}).collect(s).simplify()

EU_M_LT = EU_M_given_trigger_LT_arrival
print('E(U_M| Tr = LT) = ', EU_M_given_trigger_LT_arrival)
pprint(EU_M_given_trigger_LT_arrival)

C_MM_LT = EU_M_LT.subs({s:0})
D_MM_LT = EU_M_LT.subs({s:1})

print('C_MM_LT = ', C_MM_LT)
print('D_MM_LT = ', D_MM_LT)




Expected MM Utility when trigger event is LT arrival
E(U_M| Tr = LT) =  alphabar*q*s - alphabar*q + mubar*s + s
α̅⋅q⋅s - α̅⋅q + μ̅⋅s + s
C_MM_LT =  -alphabar*q
D_MM_LT =  mubar + 1


---
## 3.3  Combine conditional expectations to compute $EU_M$

$$ E(U_M) = (1-\beta) \, E(U_M \, | \, Tr = LT) + \beta \, E(U_M \,| \, Tr = N)  $$

From this we can compute the end points, by combining the results obtained 
above: 



$$ C := E(U_M \,|\, s= 0) = 
(1-\beta)(-\overline{\alpha} q) + \beta \left\{ h (-\gamma m)  + (1-h)(-q \overline{\mu} )\right\}
$$
This can be simplified to 

$$ C =  -\left\{\overline{\alpha}(1-\beta) + \beta \overline{\mu}\right\} q 
 - h \beta \left\{(1-\overline{\mu})  + (1- 2\overline{\mu} )q )\right\}
$$
and it is straightforward to check that this expression can be further transformed to: 

$$ C=  - \left\{q\overline{\theta} +  h  \beta(m \gamma 
- \overline{\mu} q)\right\}$$

Similarly, 
$$ D := E(U_M \,|\, s = 1) = (1-\beta)(1+\overline{\mu}) 
+ \beta(h (2\overline{\mu}-\overline{\alpha} q) + (1-h)(2\overline{\mu} ))$$

which can be simplified to: 

$$ D = (1+\overline{\mu}) - \beta (m + \overline{\alpha} q h) $$
 


In [0]:
# Combine all the conditional expectations to compute EU_M

EU_M = beta * (EU_M_news) + (1-beta)* EU_M_LT

print('======== Expected utility market maker EU_M ================= ')
print('EU_M = ', EU_M)

C = beta*C_MM_news + (1-beta)*C_MM_LT
D = beta*D_MM_news + (1-beta)*D_MM_LT

C = C.expand()
D = D.expand()

print('C = ', C)
print('D = ', D)


print('  ')
print('========  Analysing C = C0 + C1*h =============')
# Compute C = EU_M(s=0)
C = EU_M.subs({s:0})   #  C expression to be used in manipulations below

#  Decompose C in terms of the variables
#  Start with:  C = C0 + C1*h

C_poly = poly_from_expr(C,h)[0]
C0 = C_poly.coeffs()[1].collect(q).collect(alphabar)
C1 = C_poly.coeffs()[0].collect(beta).collect(q)
C1 = C1.subs({mubar:1-m})
print(' ')
print('C0 = ' , C0)
print('C1 = ', C1)

print(' ')
print('========= Analysing D =  ======================')

D = D.collect(beta)
D = D.subs(mubar, (1-m))
print('D = ' , D)




======== Expected utility market maker EU_M ================= 
EU_M =  beta*(h*(mubar*q + mubar - q + s*(-alphabar*q - mubar*q + mubar + q + 1) - 1) + mubar*(-h + 1)*(-q + s*(q + 2))) + (-beta + 1)*(alphabar*q*s - alphabar*q + mubar*s + s)
C =  alphabar*beta*q - alphabar*q + 2*beta*h*mubar*q + beta*h*mubar - beta*h*q - beta*h - beta*mubar*q
D =  -alphabar*beta*h*q + beta*mubar - beta + mubar + 1
  
========  Analysing C = C0 + C1*h =============
 
C0 =  q*(alphabar*(beta - 1) - beta*mubar)
C1 =  beta*(mubar + q*(2*mubar - 1) - 1)
 
========= Analysing D =  ======================
D =  beta*(-alphabar*h*q + mubar - 1) + mubar + 1


#  4. Dependence of utility (endpoints) on sniping probability $p$

The endpoints A, B, C and D depend on p explicitly (at least in the case of A and B) as well as implicitly through the functions g(p) and h(p): 

$$ A(p) = m\beta p g(p) \Longrightarrow \frac{dA}{dp} = m\beta\,(g(p) + p g'(p))  $$


$$ B(p) = -\overline{\alpha} q\beta p g(p) \Longrightarrow \frac{dB}{dp} = -\overline{\alpha} q \beta \,(g(p) + p g'(p))  $$

$$ C(p) = - \left\{ q \overline{\theta} + h(p)\beta(m\gamma -\overline{\mu}q)\right\} \Longrightarrow \frac{dC}{dp} = 
\frac{dC}{dh} \frac{dh}{dp}  = -\beta(m\gamma - \overline{\mu}q) \, h'(p)$$

$$ D(p) = (1+\overline{\mu}) - \beta (m+\overline{\alpha} q h(p)) 
\Longrightarrow \frac{dD}{dp} = 
\frac{dD}{dh} \frac{dh}{dp} = - \overline{\alpha} \beta q \, h'(p)  $$

Hence, to compute the required conditions, we need the values 
of these derivatives only 
for p=0 and p=1. 

In [0]:
# Define the endpoints A,B,C and D as functions of individual sniping probability p
#  as well as the corresponding derivatives 

#  The detailed computation of the functions h(p) and g(p) is discussed in 
#  another notebook.  (supplementary 2).
#  Here we just copy the results for g, h and its derivatives. 

h = ( p*H - 1 + (1-p)**H)/ (p*H)
g = h/((H-1)*p)
dg_dp = symbols('dg_dp')
dh_dp = symbols('dh_dp')

h_0 = 0
h_1 = (H-1)/H
dh_dp_0 = Rational(1,2)*(H-1)    #(H-1)/2
dh_dp_1 = 1/H

g_0 = Rational(1,2)  # 1/2
g_1 = 1/H
dg_dp_0 = Rational(-1,6)*(H-2)  # - (H-2)/6
dg_dp_1 = -(H-2)/(H*(H-1))

#  Bandit:  A -endpoint
#------------ 
print('====== BANDIT: =========')
print('')

A_p = m*beta*p*g      #  notice that A_1 = A (intercept for sure sniping)
dA_dp = m*beta*(g+p*dg_dp)

#  ISSUE: SUBSTITUTING p = 0 yields nan because formally g_p has p in denominator
#  for that reason we hardcode A_0
A_0 = 0  # A_p.subs({p:0, g:g_0}) 
A_1 = (1-mubar)*beta/H     # A_p.subs({p:1, g:g_1})

#print(A_p)
print('A(p=0) = ', A_0)
print('A(p=1) = ', A_1)
#print('Sanity check: A(p=1) should be equal to A = ', A)


dA_dp_0 = dA_dp.subs({p:0, g:g_0, dg_dp: dg_dp_0})
dA_dp_1 = dA_dp.subs({p:1, g:g_1, dg_dp: dg_dp_1})

print("A'(0) = ", dA_dp_0)
print("A'(1) = ", dA_dp_1)



#  Bandit:  B -endpoint
#-------------------------------------
B_p = -alphabar*q*beta*p*g
dB_dp = -alphabar*q*beta*(g + p* dg_dp)

B_0 = 0
B_1 = -alphabar*q*beta*g_1 # B_p.subs({p:1, g:g_1})

dB_dp_0 = dB_dp.subs({p:0, g:g_0, dg_dp: dg_dp_0})
dB_dp_1 = dB_dp.subs({p:1, g:g_1, dg_dp: dg_dp_1})

print('')
print('B(0) = ', B_0)
print('B(1) = ', B_1)
print("B'(0) = ", dB_dp_0)
print("B'(1) = ", dB_dp_1)

#------------------------------
# Market maker
#-------------------------------

print('')
print('======= MARKET MAKER ========')
print('') 

C_p = -(q*thetabar + h*beta*(m*gamma-mubar*q))
dC_dp = -beta*(m*gamma -mubar*q)* dh_dp

C_0 = C_p.subs({h:h_0})
C_1 = C_p.subs({h:h_1})
dC_dp_0  = dC_dp.subs({dh_dp:dh_dp_0})
dC_dp_0 = dC_dp_0.factor()
dC_dp_1  = dC_dp.subs({dh_dp:dh_dp_1})

print("C(0) = ", C_0)
print("C(1) = ", C_1)

print("C'(0) = ", dC_dp_0)
print("C'(1) = ", dC_dp_1)

# Endpoint D

D_p = (1+mubar) - beta*(m + alphabar*q*h)
dD_dp = - beta*alphabar*q*dh_dp

D_0 = D_p.subs({h:h_0})
D_1 = D_p.subs({h:h_1})
dD_dp_0 = - beta*alphabar*q*dh_dp_0
dD_dp_0 = dD_dp_0.factor()
dD_dp_1 = - beta*alphabar*q*dh_dp_1

print('')
print("D(0) = ", D_0)
print("D(1) = ", D_1)
print("D'(0) = ", dD_dp_0)
print("D'(1) = ", dD_dp_1)




====== BANDIT: =========

A(p=0) =  0
A(p=1) =  beta*(-mubar + 1)/H
A'(0) =  beta*(-mubar + 1)/2
A'(1) =  beta*(-mubar + 1)*((-H + 2)/(H*(H - 1)) + 1/H)

B(0) =  0
B(1) =  -alphabar*beta*q/H
B'(0) =  -alphabar*beta*q/2
B'(1) =  -alphabar*beta*q*((-H + 2)/(H*(H - 1)) + 1/H)

======= MARKET MAKER ========

C(0) =  -q*thetabar
C(1) =  -q*thetabar - beta*(H - 1)*(gamma*(-mubar + 1) - mubar*q)/H
C'(0) =  beta*(H - 1)*(gamma*mubar - gamma + mubar*q)/2
C'(1) =  -beta*(gamma*(-mubar + 1) - mubar*q)/H

D(0) =  -beta*(-mubar + 1) + mubar + 1
D(1) =  -beta*(-mubar + 1 + alphabar*q*(H - 1)/H) + mubar + 1
D'(0) =  -alphabar*beta*q*(H - 1)/2
D'(1) =  -alphabar*beta*q/H



---
# 5. Equilibrium (indifference??) utility and spread #
section 3.6



**Equilibrium/indifference spread and utility** 

$$ \begin{eqnarray}
   u^*(p)  & = & \frac{A(p)\,D(p)-B(p)\,C(p)}{(A(p)-C(p))+(D(p)-B(p))}
\end{eqnarray}$$ 

$$ s^*(p) =  \frac{A(p) - C(p)}{(A(p)-C(p))+(D(p)-B (p))}$$



In [0]:
# Computing optimal utility (U*(p))
N =  A - C + D - B
Q = A*D - B*C
#Optimal utility is the following:
u_star =( A *D - B*C)/N
print('Indifference Utility u*(p):')
print(u_star)

# Computing optimal spread (S*(p))

s_star = (A - C)/ N
print('Indifference spread s*(p) = ')
print(s_star)


Indifference Utility u*(p):
(-alphabar*beta*g*p*(-gamma + 1)*(-alphabar*q*(-beta + 1) + beta*(h*(mubar*q + mubar - q - 1) - mubar*q*(-h + 1))) + beta*g*p*(-mubar + 1)*(beta*(-alphabar*h*q + mubar - 1) + mubar + 1))/(-alphabar*beta*g*p*(-gamma + 1) + alphabar*q*(-beta + 1) + beta*g*p*(-mubar + 1) - beta*(h*(mubar*q + mubar - q - 1) - mubar*q*(-h + 1)) + beta*(-alphabar*h*q + mubar - 1) + mubar + 1)
Indifference spread s*(p) = 
(alphabar*q*(-beta + 1) + beta*g*p*(-mubar + 1) - beta*(h*(mubar*q + mubar - q - 1) - mubar*q*(-h + 1)))/(-alphabar*beta*g*p*(-gamma + 1) + alphabar*q*(-beta + 1) + beta*g*p*(-mubar + 1) - beta*(h*(mubar*q + mubar - q - 1) - mubar*q*(-h + 1)) + beta*(-alphabar*h*q + mubar - 1) + mubar + 1)


# Transition from sure to probabilistic sniping (threshold $\overline\gamma_K$ )

The transition from pure to probabilistic sniping occurs when 
 $$ \left. \frac{du^*}{dp} \right|_{(p=1)} = 0 $$

Introducing short-hand notation for the numerator and denominator for $u*(p)$  (see cell???) we get: 

 $$u^*(p)   =  \frac{A(p)\,D(p)-B(p)\,C(p)}{(A(p)-C(p))+(D(p)-B(p))} \equiv \frac{N(p)}{Q(p)}, $$

whence

$$  \frac{du^*}{dp} = \frac{N'(p)Q(p) - N(p) Q'(p)}{Q^2(p)} $$

From this we see that the transition condition can be recast as: 

$$  N'(1)Q(1) - N(1)Q'(1) = 0.   $$

We first compute all the relevant factors in the LHS of this condition. 


In [0]:
#  Incrementally building the conditions for the thresholds


N_0 = A_0*D_0 - B_0*C_0
N_1 = A_1*D_1 - B_1*C_1
Q_0 = A_0 - C_0 + D_0 - B_0
Q_1 = A_1 - C_1 + D_1 - B_1



dict_subs_numerical_vals = {alpha: alpha_val, mu : mu_val, delta :delta_val, H : H_val, gamma :gamma_val, q: q_val, 
                            beta: beta_val, mubar:mubar_val, theta :theta_val, thetabar :thetabar_val, alphabar: alphabar_val, m:m_val}

# print(N_1.subs(dict_subs_numerical_vals))

dN_dp = ( (dA_dp*D_p)+ (A_p*dD_dp)) - ((dB_dp*C_p)+(B_p*dC_dp))
dN_dp_0 =  ( (dA_dp_0*D_0)+ (A_0*dD_dp_0)) - ((dB_dp_0*C_0)+(B_0*dC_dp_0))
dN_dp_1 =  ( (dA_dp_1*D_1)+ (A_1*dD_dp_1)) - ((dB_dp_1*C_1)+(B_1*dC_dp_1))

dQ_dp = dA_dp - dC_dp + dD_dp - dB_dp
dQ_dp_0 = dA_dp_0 - dC_dp_0 + dD_dp_0 - dB_dp_0
dQ_dp_1 = dA_dp_1 - dC_dp_1 + dD_dp_1 - dB_dp_1

print('')
print('  To compute du/dp_1 we need N_1,Q_1, dN_dp_1 and dQ_dp_1 ')


print('-------------- N_1 ----------------')
N_1 = N_1.expand().cancel()
print('N_1 = ', N_1) 
print('')
#pprint(N_1)

#  Extract numerator and denominator separately

N_1_numer, N_1_denom = N_1.as_numer_denom()
print('Extracting numerator and denominator')
print('')
print('N_1 numerator: ')
print('-- number of factors = ', len(N_1_numer.factor().args)) 
print('-- expansion: ', N_1_numer.factor())
print('N_1 denominator = ') 
print('-- number of factors = ', len(N_1_denom.factor().args)) 
print('-- expansion: ', N_1_denom.factor())
print('------------------------------')


print('')
print('-------------- Q_1 ----------------')
Q_1 = Q_1.expand().cancel()
print('Q_1 = ', Q_1) 
print('')
#pprint(Q_1)

#  Extract numerator and denominator separately

Q_1_numer, Q_1_denom = Q_1.as_numer_denom()
print('Extracting numerator and denominator')
print('')
print('Q_1 numerator: ')
print('-- number of factors = ', len(Q_1_numer.factor().args)) 
print('-- expansion: ', Q_1_numer.factor())
print('Q_1 denominator = ') 
print('-- number of factors = ', len(Q_1_denom.factor().args)) 
print('-- expansion: ', Q_1_denom.factor())
print('------------------------------')



print('')
print('-------------- dN_dp_1 ----------------')
dN_dp_1 = dN_dp_1.expand().cancel()
print('dN_dp_1 = ', dN_dp_1) 
print('')
#pprint(dN_dp_1)

#  Extract numerator and denominator separately

dN_dp_1_numer, dN_dp_1_denom = dN_dp_1.as_numer_denom()
print('Extracting numerator and denominator')
print('')
print('dN_dp_1 numerator: ')
print('-- number of factors = ', len(dN_dp_1_numer.factor().args)) 
print('-- expansion: ', dN_dp_1_numer.factor())
print('dN_dp_1 denominator = ') 
print('-- number of factors = ', len(dN_dp_1_denom.factor().args)) 
print('-- expansion: ', dN_dp_1_denom.factor())
print('------------------------------')

print('')
print('-------------- dQ_dp_1 ----------------')
dQ_dp_1 = dQ_dp_1.expand().cancel()
print('dQ_dp_1 = ', dQ_dp_1) 
print('')
#pprint(dQ_dp_1)

#  Extract numerator and denominator separately

dQ_dp_1_numer, dQ_dp_1_denom = dQ_dp_1.as_numer_denom()
print('Extracting numerator and denominator')
print('')
print('dQ_dp_1 numerator: ')
print('-- number of factors = ', len(dQ_dp_1_numer.factor().args)) 
print('-- expansion: ', dQ_dp_1_numer.factor())
print('dQ_dp_1 denominator = ') 
print('-- number of factors = ', len(dQ_dp_1_denom.factor().args)) 
print('-- expansion: ', dQ_dp_1_denom.factor())
print('------------------------------')







  To compute du/dp_1 we need N_1,Q_1, dN_dp_1 and dQ_dp_1 
-------------- N_1 ----------------
N_1 =  (H*alphabar*beta**2*gamma*mubar*q - H*alphabar*beta**2*gamma*q + H*alphabar*beta**2*mubar*q**2 + H*alphabar*beta**2*mubar*q - H*alphabar*beta**2*q - H*alphabar*beta*q**2*thetabar - H*beta**2*mubar**2 + 2*H*beta**2*mubar - H*beta**2 - H*beta*mubar**2 + H*beta - alphabar*beta**2*gamma*mubar*q + alphabar*beta**2*gamma*q - alphabar*beta**2*mubar*q**2 - alphabar*beta**2*mubar*q + alphabar*beta**2*q)/H**2

Extracting numerator and denominator

N_1 numerator: 
-- number of factors =  2
-- expansion:  beta*(H*alphabar*beta*gamma*mubar*q - H*alphabar*beta*gamma*q + H*alphabar*beta*mubar*q**2 + H*alphabar*beta*mubar*q - H*alphabar*beta*q - H*alphabar*q**2*thetabar - H*beta*mubar**2 + 2*H*beta*mubar - H*beta - H*mubar**2 + H - alphabar*beta*gamma*mubar*q + alphabar*beta*gamma*q - alphabar*beta*mubar*q**2 - alphabar*beta*mubar*q + alphabar*beta*q)
N_1 denominator = 
-- number of factors =  2
-- e

## Expand as a cubic polynomial in risk aversion factor $\gamma$
 
The transition from pure to probabilistic sniping is governed by:  
$$  N'(1)Q(1) - N(1)Q'(1) = 0.   $$

From the calculations in the previous cell, we find explicit (and simple)  expressions for the denominators of the four quantities involved (we leave the numerators unspecified for now): 
 
  1.  N(1) = N_1 = N_1_numer/H^2
  2.  Q(1) = Q_1 = Q_1_numer/H 
  3.  N'(1) = dN/dp_1 = dN_dp_1_numer/((H-1)*H^2)
  4.  Q'(1) = dQ/dp_1 = dQ_dp_1_numer((H-1)*H)
 
This implies that 

   1.  N'(1)Q(1) --->  denominator = $(H-1)H^3$
   2.  N(1)Q'(1) --->  denominator = $(H-1)H^3$

The common denominator in condition gamma_K therefore is $(H-1)H^3$. From the expansions above we can conclude that both terms in gamma_K have the same denominator $(H-1)H^3$ and it suffices to add the numerators.  

Next we define the condition for the gamma_K threshold by combining the numerators: 

$$ \mbox{gamma_K_cond = dN_dp_1_numer*Q_1_numer -  N_1_numer*dQ_dp_1_numer }  $$

This turns out to be a cubic polynomial in $\gamma$ for which we need 
to identify the zero-crossings: 

$$ K(\gamma) := K_3 \gamma^3 + K_2 \gamma^2 + K_1 \gamma +K_0 = 0  $$

It turns out that we can simplify slightly by factoring out some common factors: 

$$  K_3 = -\overline{\alpha}\beta K_{30} \quad\quad 
 K_2 = \overline{\alpha}\beta K_{20} \quad\quad 
  K_1 = -\beta K_{10} \quad\quad 
 K_0  = \beta K_{00} 
 $$


In [0]:

#==========================
#  Preliminary conclusion
#===========================


# $$  N'(1)Q(1) - N(1)Q'(1) = 0.   $$

#
#  From the calculations in the previous cell, we find explicit (and simple)  expressions 
#  for the denominators of the four quantities involved: 
# 
#  N_1 = N_1_numer/H^2
#  Q_1 = Q_1_numer/H 
#  dN/dp_1 = dN_dp_1_numer/((H-1)*H^2)
#  dQ/dp_1 = dQ_dp_1_numer((H-1)*H)
# 
# This implies that 
#     N'(1)*Q(1) --->  denominator = (H-1)*H^3
#     N(1)*Q'(1) --->  denominator = (H-1)*H^3
#
#  The common denominator therefore is:   H^2(1-H)^2
#  We can therefore rebuild the condition for gamma_K as follows: 
#  
#  gamma_K_cond = N'(1)*Q(1) - N(1)*Q'(1) = 0
# 
#  From the expansions above we can conclude that both terms in gamma_K 
#  have the same denominator (H-1)*H^3 and it suffices to add the appropriate numerators

gamma_K_cond = dN_dp_1_numer*Q_1_numer -  N_1_numer*dQ_dp_1_numer

gamma_K_cond = gamma_K_cond.subs({q:gamma-1})

# Expand as a polynomial in gamma 

poly_gamma_K_cond = poly_from_expr(gamma_K_cond,gamma)[0]

#  poly_gamma_K_cond = K3*gamma^3 + K2*gamma^2 + K1*gamma + K0

# Produce a list of cofficients (list[0] corresponds to highest degree!)
gamma_K_cond_coeffs = poly_gamma_K_cond.all_coeffs()

print('---------------')
for cc in  gamma_K_cond_coeffs:
  print(cc.factor())
print('----------------')  

#  Zoom in on the coefficient of gamma^3  (i.e. gamma_K_cond_coeffs[0])

K3 = gamma_K_cond_coeffs[0].factor()
K2 = gamma_K_cond_coeffs[1].factor()
K1 = gamma_K_cond_coeffs[2].factor()
K0 = gamma_K_cond_coeffs[3].factor()

print('K3  = ' ,K3)
print('K2  = ' ,K2)
print('K1  = ' ,K1)
print('K0  = ' ,K0)


#  From the listing of the K-coefficients we see that we can further factorize: 
#  K3 = -alphabar*beta*K30
#  K2 =  alphabar*beta*K20
#  K1 = -beta*K10
#  K0 =  beta*K00

#  Extract the corresponding factors: 
K30 = -(K3/(alphabar*beta)).cancel()
K20 = (K2/(alphabar*beta)).cancel()
K10  = -(K1/beta).cancel()
K00  = (K0/beta).cancel()

print('')
print('K30 = ', K30)
print('K20 = ', K20)
print('K10 = ', K10)
print('K00 = ', K00)





---------------
-alphabar*beta*(2*H**2*alphabar*beta**2*mubar - H**2*alphabar*beta**2 + 4*H**2*beta**2*mubar**2 - 4*H**2*beta**2*mubar + H**2*beta**2 - 4*H**2*beta*mubar*thetabar + 2*H**2*beta*thetabar + H**2*thetabar**2 - 6*H*alphabar*beta**2*mubar + 3*H*alphabar*beta**2 - 8*H*beta**2*mubar**2 + 8*H*beta**2*mubar - 2*H*beta**2 + 4*H*beta*mubar*thetabar - 2*H*beta*thetabar + 4*alphabar*beta**2*mubar - 2*alphabar*beta**2 + 4*beta**2*mubar**2 - 4*beta**2*mubar + beta**2)
alphabar*beta*(4*H**2*alphabar*beta**2*mubar - H**2*alphabar*beta**2 + 10*H**2*beta**2*mubar**2 - 9*H**2*beta**2*mubar + 2*H**2*beta**2 + 4*H**2*beta*mubar**2 - 9*H**2*beta*mubar*thetabar + 2*H**2*beta*mubar + 3*H**2*beta*thetabar - 2*H**2*beta - H**2*mubar*thetabar + 3*H**2*thetabar**2 - H**2*thetabar - 12*H*alphabar*beta**2*mubar + 3*H*alphabar*beta**2 - 18*H*beta**2*mubar**2 + 15*H*beta**2*mubar - 3*H*beta**2 - 4*H*beta*mubar**2 + 8*H*beta*mubar*thetabar - 2*H*beta*mubar - 2*H*beta*thetabar + 2*H*beta + 8*alphabar*bet

# Properties of the cubic polynomial in gamma

$$ K(\gamma) = K_3 \gamma^3 + K_2\gamma^2 + K_1\gamma + K_0  $$

##  $K_3 < 0$

In this section we first show that $K_3 < 0$  which implies that 

$$ \lim_{\gamma\rightarrow \infty} K(\gamma) = -\infty  $$

Since $$K_3 = -\overline{\alpha}\beta K_{30}  $$

we have to show that $K_{30}>0$. 

To this end we recast $K_{30}$ as a quadratic polynomial in $\beta$:

$$  K_{30}  = K_{302}\beta^2 +   K_{301}\beta +  K_{300}$$

Since $0 \leq \beta \leq 1$ it suffices to show that $K_{30} > 0$ over that 
$\beta$-range.  This follows from the observations: 

  1. $K_{302} > 0 $ so the $\beta$-quadratic is convex; 
  2. $K_{30}(\beta=0) = K_{300}> 0$
  3. The minimum of the parabola is located at negative $\beta$-values;  this follows from the fact that $K_{302}> 0$  and the minimum is achieved for 
  $\beta$-value equal to $-K_{301}/(2K_{302}) < 0$.

These observations guarantee that the $\beta$-polynomial is positive 
for positive $\beta$-values. 


In [0]:
#  we have to prove that K30 > 0  (to show that K3 < 0)

#  Sanity check:  try some numerical values: 

# print('Numerical check:  K3 = ', K3.subs(dict_subs_numerical_vals))


#  Express K30 as (quadratic) polynomial 

print('K30 as polynomial in beta')
print('')
poly_K30 = poly_from_expr(K30,beta)[0]
print(poly_K30)

print('')
print('Coefficients of K30 in beta')

print('')
K302 = (poly_K30.all_coeffs()[0]).factor()
K301 = (poly_K30.all_coeffs()[1]).factor()
K300 = (poly_K30.all_coeffs()[2]).factor()

print('K30-coef of beta^2 = K302 = ', K302)
print('K30-coef of beta^1 = K301 = ', K301)
print('K30-coef of beta^0 = K300 = ', K300)

#  Since mubar < 1/2  it follows that (2*mubar-1) < 0.
#  Simlarly:  alphabar + mubar < 1/2   (2*alphabar + 2*mubar -1) < 0  

print('')
print('Clearly K300 > 0')
print('Since (2*mubar-1)< 0 , it follows that K301 > 0')
print(' ')
print('Also: if we denote  R =  (H*alphabar + 2*H*mubar - H - 2*alphabar - 2*mubar + 1)')
print('then by adding H*alphabar, we get')

R = H*alphabar + 2*H*mubar - H - 2*alphabar - 2*mubar + 1
RR = R+alphabar*H 
print(RR.factor())
print('R < R+ H*alphabar = (2H-1) = ', RR.factor()) 

print('Since 2*alphabar+2*mubar-1 < 0, it follows that the RHS < 0  and hence K302 > 0 ')

print('')
print('')
print('Numerial sanity check: for the values in the dictionary:')
print(K302.subs(dict_subs_numerical_vals))
print(K301.subs(dict_subs_numerical_vals))
print(K300.subs(dict_subs_numerical_vals))



K30 as polynomial in beta

Poly((2*H**2*alphabar*mubar - H**2*alphabar + 4*H**2*mubar**2 - 4*H**2*mubar + H**2 - 6*H*alphabar*mubar + 3*H*alphabar - 8*H*mubar**2 + 8*H*mubar - 2*H + 4*alphabar*mubar - 2*alphabar + 4*mubar**2 - 4*mubar + 1)*beta**2 + (-4*H**2*mubar*thetabar + 2*H**2*thetabar + 4*H*mubar*thetabar - 2*H*thetabar)*beta + H**2*thetabar**2, beta, domain='ZZ[H,alphabar,mubar,thetabar]')

Coefficients of K30 in beta

K30-coef of beta^2 = K302 =  (H - 1)*(2*mubar - 1)*(H*alphabar + 2*H*mubar - H - 2*alphabar - 2*mubar + 1)
K30-coef of beta^1 = K301 =  -2*H*thetabar*(H - 1)*(2*mubar - 1)
K30-coef of beta^0 = K300 =  H**2*thetabar**2

Clearly K300 > 0
Since (2*mubar-1)< 0 , it follows that K301 > 0
 
Also: if we denote  R =  (H*alphabar + 2*H*mubar - H - 2*alphabar - 2*mubar + 1)
then by adding H*alphabar, we get
(H - 1)*(2*alphabar + 2*mubar - 1)
R < R+ H*alphabar = (2H-1) =  (H - 1)*(2*alphabar + 2*mubar - 1)
Since 2*alphabar+2*mubar-1 < 0, it follows that the RHS < 0  and henc

##  $K(\gamma=1) = K_3+K_2+K_1+K_0 > 0$

It turns out that 

$$ K_3+K_2+K_1+K_0 = \beta H^2 (1-\overline{\mu})(\beta\overline{\mu} - \beta + \overline{\mu} + 1)^2  $$

which is positive since $\overline{\mu} < 1$.




In [0]:
#  Show that K3+K2+K1+K0 > 0

K_at_1 = K0+K1+K2+K3

K_at_1 = K_at_1.expand().factor()

print('K(1) = K0+K1+K2+K3 =', K_at_1)


K(1) = K0+K1+K2+K3 = -H**2*beta*(mubar - 1)*(beta*mubar - beta + mubar + 1)**2


## There is only one zero-crossing for which $\gamma>1$  

Finally, we show that the cubic polynomial $K(\gamma)$ has only one zero-crossing for which $\gamma>1$. 
This then proves that the threshold $\overline{\gamma}_K$ is unique. 

To this end we prove that the 2nd derivative $K''(1) < 0$  showing 
that the cubic polynomial is concave right from $\gamma=1$. 

We first show that: 

$$ K''(1) = -2\overline{\alpha} \beta \left\{ W_2 \beta^2 + W_1\beta +W_0\right\}   $$

It turns out that the coefficients $W_0$ and $W_2$ are easily shown 
to be positive: 

  1. $ W_2 =  (H - 2)(H - 1)(1-\overline{\mu})(1-2(\overline{\alpha} + \overline{\mu}))  > 0 $

  2. $W_0 =  H^2\overline{\theta}(1+\overline{\mu} ) > 0 $


The $W_1$-coefficient requires more work. It turns out that 

$$ W_1 = -H W_{11} = -H(W_{111} \overline{\theta} + W_{110})  $$ where 

  1. $W_{111} = -(3H - 4)(1 - \overline{\mu}) < 0  $
  2. $ W_{110} =  - 2(H - 1)(\overline{\mu} + 1)(1-2\overline{\mu})  < 0 $

Since the linear function in $\overline{\theta}$ has negative slope and 
negative intercept, it has negative value for $\overline{\theta}$.  As a consequence we conclude that also $W_1 > 0$.

From the fact that $W_0, W_1, W_2 > 0$ it follows that $K''(1) < 0$  which proves 
that the threshold $\overline{\gamma}_K$ exists and is unique. 

In [0]:
#  Compute  K'(gamma) = 

#dK_dgamma = diff(K,gamma)

dK_dgamma_at_0 = K1

print(K1.factor())

print(K2.factor())

print('')
d2K_at_1 = 6*K3 + 2*K2
d2K_at_1 = d2K_at_1.expand().simplify().factor()

W = (d2K_at_1/(-2*alphabar*beta)).cancel()

print('W = ',W)

print(d2K_at_1)

poly_W = poly_from_expr(W,beta)[0]
print(poly_W)

print('===================')
print(d2K_at_1.subs(dict_subs_numerical_vals))
print('')

print(K1.subs(dict_subs_numerical_vals))

print(K2.subs(dict_subs_numerical_vals))

poly_K20 = poly_from_expr(K20,beta)[0]


print('')
print('Coefficients of K20 in beta')

print('')
W2 = (poly_W.all_coeffs()[0]).factor()
W1 = (poly_W.all_coeffs()[1]).factor()
W0 = (poly_W.all_coeffs()[2]).factor()

print('W-coef of beta^2 = W2 = ', W2)
print('W-coef of beta^1 = W1 = ', W1.factor())
print('W-coef of beta^0 = W0 = ', W0)

print('')
print('Numerical values ' )
print('W0 = ', W0.subs(dict_subs_numerical_vals)) 
print('W1 = ', W1.subs(dict_subs_numerical_vals))
print('W2 = ', W2.subs(dict_subs_numerical_vals))

print('')

print('Numerical tests seem to suggest that all three W coefficients are positive')
print('As a consequence W is positive and d2K_at_1 is negative')
print('This means that the cubic polynomial is in its concave part -- meaning there is only one zero-crossing greater than 1')

print(D.subs(dict_subs_numerical_vals))

print(D.expand().simplify().factor())

print('W1 is the problem:  W1 = -H*W11')

W11 = (W1/(-H)).cancel()

print('W11 = ', W11.collect(thetabar))

print('We have to show that W11 < 0')

poly_W11 = poly_from_expr(W11,thetabar)[0]

print(poly_W11)

print('Coeff of thetabar^1 = ' , poly_W11.all_coeffs()[0].factor())
print('Coeff of thetabar^0 = ' , poly_W11.all_coeffs()[1].factor())

print(W11.subs(dict_subs_numerical_vals))


# plot the transition thresholds
# from sympy import plot
#plt= plot(gamma_K_cond,(gamma, 0,10),line_color ='b',show=False)
        #title = 'Transition Treshold \n red line: gamma bar_L \n blue line:gamma bar_K ', xlabel = 'Gamma',ylabel='', xlim=[0,10], ylim=[-.3,.3])
#plt.extend(plot(gamma_L_cond,(beta,0,1),line_color ='r',show=False))
#plt.show()




-beta*(2*H**2*alphabar**2*beta**2*mubar + H**2*alphabar**2*beta**2 + 6*H**2*alphabar*beta**2*mubar**2 - 2*H**2*alphabar*beta**2*mubar - H**2*alphabar*beta**2 + 4*H**2*alphabar*beta*mubar**2 - 6*H**2*alphabar*beta*mubar*thetabar + 4*H**2*alphabar*beta*mubar - 2*H**2*alphabar*mubar*thetabar + 3*H**2*alphabar*thetabar**2 - 2*H**2*alphabar*thetabar + H**2*beta*mubar**2*thetabar - 2*H**2*beta*mubar*thetabar + H**2*beta*thetabar + H**2*mubar**2*thetabar - H**2*thetabar - 6*H*alphabar**2*beta**2*mubar - 3*H*alphabar**2*beta**2 - 10*H*alphabar*beta**2*mubar**2 + 2*H*alphabar*beta**2*mubar + 2*H*alphabar*beta**2 - 4*H*alphabar*beta*mubar**2 + 4*H*alphabar*beta*mubar*thetabar - 4*H*alphabar*beta*mubar + 2*H*alphabar*beta*thetabar + 4*alphabar**2*beta**2*mubar + 2*alphabar**2*beta**2 + 4*alphabar*beta**2*mubar**2 - alphabar*beta**2)
alphabar*beta*(4*H**2*alphabar*beta**2*mubar - H**2*alphabar*beta**2 + 10*H**2*beta**2*mubar**2 - 9*H**2*beta**2*mubar + 2*H**2*beta**2 + 4*H**2*beta*mubar**2 - 9*H**

#  Condition on polynomial  $L(\gamma)$

When risk-aversion is too severe, even probabilistic sniping won't pay.  The value for which this happens occurs when $N'(0) \equiv dN/dp(p=0) = 0$

In [0]:
#  Express dN/dp(p=0) as a polynomial in q = \gamma-1: 



poly_dN_dp_0 = poly_from_expr(dN_dp_0,q)[0]

print("N'(0) = dN/dp(p=0) is a polynomial of degree  ", degree(poly_dN_dp_0,gen=q),  " in q = gamma-1")

print('')
print('Polynomial coefficients (in q):')
L2 = poly_dN_dp_0.all_coeffs()[0].factor()
L1 = poly_dN_dp_0.all_coeffs()[1].factor()
L0 = poly_dN_dp_0.all_coeffs()[2].factor()

print('Coef of q^2 = L2 = ', L2)
print('Coef of q^1 = L1 = ', L1)
print('Coef of q^0 = L0 = ', L0)



N'(0) = dN/dp(p=0) is a polynomial of degree   2  in q = gamma-1

Polynomial coefficients (in q):
Coef of q^2 = L2 =  -alphabar*beta*thetabar/2
Coef of q^1 = L1 =  0
Coef of q^0 = L0 =  -beta*(mubar - 1)*(beta*mubar - beta + mubar + 1)/2
